## Conditional Random Field Model (Window 1)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#Custom models
from prepro import readfile, get_sentence, is_number, extract_words,get_label,partial_tags

#Model
from sklearn_crfsuite import CRF

#Evalulation
from sklearn.model_selection import cross_val_predict
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn_crfsuite.metrics import flat_f1_score

In [2]:
#import data from my github repo
#test = readfile("test.txt")
data =readfile("train.txt")
print("Number of Sentences in dataset:",len(data))

Number of Sentences in dataset: 8961


In [3]:
train,test=train_test_split(data,train_size=.8, random_state=0)

## Orthographic Feature Engineering

In [4]:
#Special Features:
def count_vowel(word):
    '''
    Function returns the number of vowels in token
    '''
    return sum(list(map(word.lower().count, "aeiou")))

def dash(word):
    '''
    The Function returns whether or not the token contains a dash
    '''
    return 1 if "-" in word else 0

def count_consonants(word):
    '''
    The Function returns the number of consonants in a token
    '''
    vowels="aeiou"
    return sum(i not in vowels for i in word)

In [5]:
def word2features(sentence,i):
    word = sentence[i]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'Prefix_2': word[:2],
        'Prefix_3': word[:3], 
        'word.isupper()': word.isupper(),
        'word.islower()': word.islower(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'word.isalpha()': word.isalpha(),
        'word.isalnum()': word.isalnum(), 
        "vowels": count_vowel(word),
        "dash": dash(word),
        "consonants": count_consonants(word),
        'Suffix_2': word[-2:],
        'Suffix_3': word[-3:],
        
    }
    if i > 0:
        '''
        Get the token before in the sentence
        '''
        word1 = sentence[i-1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word Prefix_2': word1[:2],
            '-1:word Prefix_3': word1[:3], 
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:word.islower()': word1.lower(),
            '-1:word.isdigit()': word1.isdigit(),
            '-1:word.isalpha()': word1.isalpha(),
            '-1:word.isalnum()': word1.isalnum(), 
            "-1:word.vowels": count_vowel(word1),
            "-1:word.dash": dash(word1),
            "-1:word.consonants": count_consonants(word1),
            '-1:word.Suffix_2': word1[-2:],
            '-1:word.Suffix_3': word1[-3:],
        })
    else:
        features['START'] = True
        
    if i < len(sentence)-1:
        '''
        Get the token after in the sentence
        '''
        word1 = sentence[i+1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.Prefix_2': word1[:2],
            '+1:word.Prefix_3': word1[:3], 
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:word.islower()': word1.lower(),
            '+1:word.isdigit()': word1.isdigit(),
            '+1:word.isalpha()': word1.isalpha(),
            '+1:word.isalnum()': word1.isalnum(), 
            "+1:word.vowels": count_vowel(word1),
            "+1:word.dash": dash(word1),
            "+1:word.consonants": count_consonants(word1),
            '+1:word.Suffix_2': word1[-2:],
            '+1:word.Suffix_3': word1[-3:],
        })
    else:
        features['END'] = True
    
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def get_all_sentences(dataset):
    sentences=[]
    for i in range(len(dataset)):
        sentences.append(get_sentence(dataset,i+1))
    return sentences

def get_all_labels(dataset):
    labels=[]
    for i in range(len(dataset)):
        #labels.append(partial_tags(get_label(dataset,i+1)))
        labels.append(get_label(dataset,i+1))
    return labels

In [6]:
#get the first sentence
sent = get_sentence(train,1)
#label = partial_tags(get_label(train,1))
label = get_label(train,1)

#print out first sentence
print(sent)
print(label)

['.', '3', ',', '9', ',', '15', 'Minister', 'van', 'Omgewingsake', 'en', 'Toerisme', '.']
['OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'B-PERS', 'I-PERS', 'I-PERS', 'I-PERS', 'I-PERS', 'OUT']


In [7]:
#Applt feature engineering
train_sents = get_all_sentences(train)
train_labels = get_all_labels(train)
test_sents = get_all_sentences(test)
test_labels = get_all_labels(test)

X_train = [sent2features(s) for s in train_sents]
y_train = train_labels
X_test = [sent2features(s) for s in test_sents]
y_test = test_labels


sub_labels=list(set([item for sublist in train_labels for item in sublist]))
sub_labels.remove("OUT")


## Condtional Random Field Model


In [8]:
%%time
crf4 = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.2,
          max_iterations=50,
          all_possible_transitions=False)

#training model
crf4.fit(X=X_train, y=y_train)

#generate predictions
pred = crf4.predict(X_train)

#generate report on entire model
report = flat_classification_report(y_pred=pred, y_true=y_train,labels=sub_labels)
print(report)

score=flat_f1_score(y_pred=pred, y_true=y_train,average='micro',labels=sub_labels)
print("F1=%.2f"%(score*100))

              precision    recall  f1-score   support

       I-LOC       0.97      0.94      0.96       288
      B-MISC       0.95      0.91      0.93      5264
      B-PERS       0.97      0.97      0.97      1627
       B-LOC       0.97      0.96      0.96      1478
      I-MISC       0.96      0.93      0.94      5371
       B-ORG       0.91      0.95      0.93      2831
       I-ORG       0.90      0.98      0.94      2164
      I-PERS       0.98      0.98      0.98      1715

   micro avg       0.94      0.94      0.94     20738
   macro avg       0.95      0.95      0.95     20738
weighted avg       0.95      0.94      0.94     20738

F1=94.42
Wall time: 30.8 s


# Predict on Test dataset

In [9]:
#prediction with best performaning model
pred = crf4.predict(X_test)

#generate report on entire model
report = flat_classification_report(y_pred=pred, y_true=y_test,labels=sub_labels)
print(report)

score=flat_f1_score(y_pred=pred, y_true=y_test,average='micro',labels=sub_labels)
print("F1=%.2f"%(score*100))

              precision    recall  f1-score   support

       I-LOC       0.61      0.46      0.52        50
      B-MISC       0.81      0.72      0.76      1267
      B-PERS       0.86      0.77      0.82       412
       B-LOC       0.87      0.75      0.81       340
      I-MISC       0.86      0.68      0.76      1357
       B-ORG       0.74      0.78      0.76       726
       I-ORG       0.61      0.81      0.69       551
      I-PERS       0.83      0.81      0.82       440

   micro avg       0.79      0.74      0.76      5143
   macro avg       0.77      0.72      0.74      5143
weighted avg       0.80      0.74      0.76      5143

F1=76.19
